In [1]:
# This is the template for the submission. If you want, you can develop your algorithm in a regular Python script and copy the code here for submission.

# Team members (e-mail, legi):
# zhisun@ethz.ch, 22-958-227
# enjcao@ethz.ch, 22-942-700
# yifzhou@ethz.ch, 22-940-381

In [2]:
import numpy as np
import pandas as pd

from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset

import os
from os import listdir
from os.path import isfile, join


import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from scipy.signal import welch
from scipy.fftpack import fft

from Lilygo.Recording import Recording, data_integrity
from Lilygo.Dataset import Dataset

import joblib

In [ ]:
# Get the path of all traces
dir_traces = '/kaggle/input/mobile-health-2023-path-detection/data/test'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()

In [ ]:
def get_highpass(data):
    filtered_data = np.zeros_like(data)  # filtered_data
    alpla = [1, -1.905384612118461, 0.910092542787947]
    beta = [0.953986986993339, -1.907503180919730, 0.953986986993339]

    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

# This funciton aims to realize a high-pass filter with cutoff frequency = 5 Hz. Because according to massive amounts of data, the general 
# maximum frequency of human walking is about 5 Hz
def get_lowpass(data):
    filtered_data = np.zeros_like(data)  # filtered_data
    alpla = [1, -1.80898117793047, 0.827224480562408]
    beta = [0.096665967120306, -0.172688631608676, 0.095465967120306]

    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

def preprocess_and_extract_features(trace, window_size=60, sampling_rate=50):
    """
    Preprocess the data and extract features from the 3D accelerometer, gyroscope, and magnetometer data.

    Args:
    trace (Lilygo.Recording.Recording): Object containing the raw data with accelerometer data stored in lists (e.g. trace.data['ax'])
    window_size (int): The window size in seconds for splitting the data
    sampling_rate (int): The sampling rate of the data in Hz

    Returns:
    pd.DataFrame: A DataFrame with the extracted features and location labels
    """
    # Read data from trace
    # To-Do: filter raw data with implemented function
    ax = get_lowpass(get_highpass(trace.data['ax'].values))
    ay = get_lowpass(get_highpass(trace.data['ay'].values))
    az = get_lowpass(get_highpass(trace.data['az'].values))
    
    '''gx = trace.data['gx'].values
    gy = trace.data['gy'].values
    gz = trace.data['gz'].values
    
    mx = trace.data['mx'].values
    my = trace.data['my'].values
    mz = trace.data['mz'].values'''
    
    # Compute the length of each window in samples
    window_samples = window_size * sampling_rate
    
    # Compute the number of windows in the recording
    num_windows = len(ax) // window_samples

    # Initialize lists for storing extracted features and location labels
    features = []
    loc_labels = []

    # Helper function to compute the magnitude of a vector
    magnitude = lambda vec: np.sqrt(np.sum(vec**2, axis=1))

    for i in range(num_windows):
       
        # Extract the accelerometer, gyroscope, and magnetometer data for the current window
        acc_data = np.array([ax[i*window_samples:(i+1)*window_samples],
                             ay[i*window_samples:(i+1)*window_samples],
                             az[i*window_samples:(i+1)*window_samples]]).T
        '''gyro_data = np.array([gx[i*window_samples:(i+1)*window_samples],
                              gy[i*window_samples:(i+1)*window_samples],
                              gz[i*window_samples:(i+1)*window_samples]]).T
        mag_data = np.array([mx[i*window_samples:(i+1)*window_samples],
                             my[i*window_samples:(i+1)*window_samples],
                             mz[i*window_samples:(i+1)*window_samples]]).T'''
        
        '''figure,ax = plt.subplots(3, 1, figsize=(10, 6))
        ax[0].plot(acc_data[:,0])
        ax[0].set_ylabel('ax')
        ax[1].plot(acc_data[:,1])
        ax[1].set_ylabel('ay')
        ax[2].plot(acc_data[:,2])
        ax[2].set_ylabel('az')'''

        # Compute magnitudes
        acc_magnitude = np.sqrt(np.sum(acc_data**2, axis=1))
        #gyro_magnitude = magnitude(gyro_data)
        #mag_magnitude = magnitude(mag_data)

        # Calculate features (according to https://www.sciencedirect.com/science/article/pii/S1574119211001222)
        '''ax_amp = acc_data[np.argmax(acc_data[:,0]), 0] - acc_data[np.argmin(acc_data[:,0]), 0]
        ay_amp = acc_data[np.argmax(acc_data[:,1]), 1] - acc_data[np.argmin(acc_data[:,1]), 1]
        az_amp = acc_data[np.argmax(acc_data[:,2]), 2] - acc_data[np.argmin(acc_data[:,2]), 2]'''
        
        ax_amp = abs(np.mean(acc_data[:,0]))
        ay_amp = abs(np.mean(acc_data[:,1]))
        az_amp = abs(np.mean(acc_data[:,2]))
        a_amp_list = [ax_amp, ay_amp, az_amp]
        a_amp_list.sort() # Sorting list of numbers in ascending
        #print('ax_amp:',ax_amp,'ay_amp:',ay_amp,'az_amp',az_amp)
        A = a_amp_list[2] #Feature A: the maximum amplitude among all dimensions (represents motion range for location)
        B = a_amp_list[2]/a_amp_list[1] # Feature B and C: ratio of the maximum amplitudes in different axes (represents DoF in movement for location)
        C = a_amp_list[2]/a_amp_list[0]
        #print('A:',A,'B:',B,'C',C)

        # Calculate the energy and entropy of acc_mag in the frequency domain (D and F)
        freq, Pxx = welch(acc_magnitude, fs=sampling_rate) # use of the fast Fourier transform for the estimation of power spectra
        #plt.plot(freq,Pxx)
        
        D = np.max(Pxx) # Feature D: the maximum energy captured by the accelerator
        F = np.sum(Pxx) # Feature F: the overall energy captured by the accelerator
        norm_Pxx = Pxx / F # normalize the power spectrum
        E = entropy(norm_Pxx) # Feature E: normalized information entropy of the discrete FFT component magnitudes

   
        # Append the features to the list
        features.append([A, B, C, D, E, F])
        
        # Calculate the timestamp for the current window as the median of the timestamps (not necessary for location)
        # timestamp = np.median(trace.timestamp[i*window_samples:(i+1)*window_samples]) 

        # Determine the location label for the current window based on the timestamp
        loc_label = trace.labels.get('board_loc')

        # Append the label to the labels list
        loc_labels.append(loc_label)
    #plt.show()

    # Create a DataFrame with the extracted features and location labels
    features_df = pd.DataFrame(features, columns=['A', 'B', 'C', 'D', 'E', 'F'])
    features_df['loc_label'] = loc_labels
    #print(features_df)

    return features_df

## Load trained model

In [ ]:
# Location
location_model = joblib.load('./trained_models/location_svm_model.joblib')

In [ ]:
def get_watch_loc(trace):
    # Get features of data
    features_df = preprocess_and_extract_features(trace)

    # Prepare data for classification
    scaler = StandardScaler()
    X_test = scaler.fit_transform(features_df)

    # Predict the location with loaded model
    y_pred = location_model.predict(X_test)
    y_pred = np.squeeze(y_pred)
    y_final = np.argmax(np.bincount(y_pred.astype(int)))
    
    return y_final

In [ ]:
def get_path_idx(trace):
    return 0

In [ ]:
def get_step_count(trace):
    return 0

In [ ]:
def get_activity(trace):
    stand, walk, run, cycle = 0, 0, 0, 0
    return stand, walk, run, cycle

In [ ]:
# Loop through all traces and calculate the step count for each trace
solution_file = []
for filename in filenames:
    trace = Recording(filename, no_labels=True, mute=True)
    categorization_results = {'watch_loc': 0, 'path_idx': 0, 'step_count': 0, 'stand': 0, 'walk': 0, 'run': 0, 'cycle': 0}

    #
    # Your algorithm goes here
    # You can access the variable 'watch_loc' in the dictionary 'categorization_results' for example with
    # categorization_results['watch_loc'] = 1
    # Make sure, you do not use the gps data and are tolerant for missing data (see task set).
    # Your program must not crash when single smartphone data traces are missing.
    #

    stand, walk, run, cycle = get_activity(trace)
    categorization_results['watch_loc'] = get_watch_loc(trace)
    categorization_results['path_idx'] = get_path_idx(trace)
    categorization_results['step_countstep_count'] = get_step_count(trace)
    categorization_results['stand'], categorization_results['walk'], categorization_results['run'], categorization_results['cycle'] = get_activity(trace)


    # Append your calculated results and the id of each trace and category to the solution file
    trace_id = ''.join([*filename][-8:-5])
    for counter_label, category in enumerate(categorization_results):
        solution_file.append([trace_id + f'_{counter_label+1}', categorization_results[category]])


In [ ]:
# Write the detected step counts into a .csv file to then upload the .csv file to Kaggle
# When cross-checking the .csv file on your computer, we recommend using the text editor and NOT excel so that the results are displayed correctly
# IMPORTANT: Do NOT change the name of the columns ('Id' and 'Category') of the .csv file
submission_file_df = pd.DataFrame(np.asarray(solution_file), columns=['Id', 'Category'])
submission_file_df.to_csv('/kaggle/working/submission.csv', header=['Id', 'Category'], index=False)